# Recommender Code
클래식 추천자 튜토리얼은 movielens 데이터 세트를 사용.  
다른 알고리즘에 MNIST 데이터 세트를 사용하는 것과 유사. 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()

Collaborative filtering을 통해 우리는 많은 사용자로부터 선호도 또는 취향 정보를 수집하여 사용자의 관심사에 대한 예측(필터링)을 수행. 기본 가정은 *사용자 A*가 문제에 대해 *사용자 B*와 ***동일한 의견***을 가지고 있다면, A는 사용자의 x에 대한 의견을 무작위로 선택하는 것보다 다른 문제 x에 대해 B의 의견을 가질 가능성이 더 높음.

아래 이미지 (Wikipedia에서 가져온)는 Collaborative filtering의 예를 보여줌. 처음에는 사람들이 다른 항목 (예:비디오, 이미지, 게임)을 평가함. 그런 다음 시스템은 아직 평가되지 않은 항목에 대한 사용자 평가를 예측. 새로운 예측은 활성 사용자와 비슷한 평가를 가진 다른 사용자의 기존 평가를 기반으로 함. 이미지에서 시스템은 사용자가 비디오를 좋아하지 않을 것이라고 예측함.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Machine Learning 용 Spark MLlib 라이브러리는 Alternating Least Squares을 사용하여 Collaborative filtering 구현을 제공. MLlib의 구현에는 다음 매개 변수가 있음.

* numBlocks는 계산을 병렬화하는 데 사용되는 블록 수입니다 (자동 구성하려면 -1로 설정).
* rank는 모델의 잠재 요인 수.
* iterations는 실행할 반복 횟수.
* lambda는 ALS에서 정규화 매개 변수를 지정.
* implicitPrefs는 명시적 피드백 ALS 변형을 사용할지 또는 암시적 피드백 데이터에 적합한 변형을 사용할지 지정.
* alpha는 선호도 관찰에서 기준선 신뢰도를 관리하는 ALS의 암시 적 피드백 변형에 적용 할 수있는 매개 변수.

In [6]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS #Alternating Least Squares
from configuration import make_engine
import pandas as pd

In [11]:
engine = make_engine()
data = pd.read_sql('SELECT * FROM movielens_ratings', con=engine)
data = spark.createDataFrame(data)

In [15]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [16]:
data.describe().show()

+-------+-----------------+------------------+------------------+
|summary|          movieId|            rating|            userId|
+-------+-----------------+------------------+------------------+
|  count|             1501|              1501|              1501|
|   mean|49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.93703406508901|1.1872761661248035| 8.591040424293263|
|    min|                0|               1.0|                 0|
|    max|               99|               5.0|                29|
+-------+-----------------+------------------+------------------+



모델의 성능을 평가하기 위해 분할을 수행할 수 있지만 추천 시스템이 일부 주제에 대해 실제로 얼마나 잘 작동하는지 결정적으로 파악하기가 매우 어려움. 특히 주관성이 관련된 경우 추천 시스템이 달리 제안할 수 있지만 스타워즈를 좋아하는 모든 사람이 스타트렉을 좋아하지는 않을 것.

In [17]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [18]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')
model = als.fit(training)

이제 모델의 성능을 살펴 보자!

In [19]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [20]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   4.0|    12|  1.1066724|
|     31|   1.0|    13|  1.4739949|
|     31|   1.0|     4|   2.067967|
|     31|   2.0|    25|-0.86705685|
|     85|   1.0|    26|   4.303054|
|     85|   1.0|    12|  6.2194014|
|     85|   1.0|    15|  0.7440838|
|     85|   1.0|     2|  2.1127574|
|     65|   2.0|     5|  3.2382467|
|     53|   2.0|    19|  2.5945947|
|     53|   1.0|    23| -2.2397356|
|     53|   1.0|     7|  2.5840058|
|     78|   1.0|    13| 0.49412483|
|     78|   1.0|    19|  0.6074424|
|     78|   1.0|    24|-0.16739936|
|     78|   1.0|    11| 0.56174606|
|     81|   1.0|    19|-0.03052605|
|     81|   1.0|    15| 0.69937754|
|     28|   5.0|    18|-0.31741253|
|     76|   1.0|    20|  1.7333409|
+-------+------+------+-----------+
only showing top 20 rows



In [21]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
rmse = evaluator.evaluate(predictions)
print('Root-mean-square error = {}'.format(rmse))

Root-mean-square error = 1.7818174059630163


이제 모델이 생겼으니 실제로 사용자에게 추천을 어떻게 제공 하는가?
테스트 데이터로했던 것과 같은 방식임

In [24]:
single_user = test.filter(test['userId'] == 11).select(['movieId', 'userId'])

In [25]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     10|    11|
|     21|    11|
|     25|    11|
|     39|    11|
|     45|    11|
|     50|    11|
|     51|    11|
|     66|    11|
|     71|    11|
|     77|    11|
|     78|    11|
|     86|    11|
|     89|    11|
|     94|    11|
|     97|    11|
+-------+------+



In [26]:
recomendation = model.transform(single_user)

In [27]:
recomendation.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     51|    11|  4.5612383|
|     71|    11|  3.6289947|
|      0|    11|  2.5576847|
|     25|    11|  2.3242495|
|     21|    11|  1.3120706|
|     94|    11|  1.1792042|
|     39|    11|  1.0927076|
|     89|    11|  1.0346172|
|     66|    11| 0.77329224|
|     97|    11| 0.72503513|
|     78|    11| 0.56174606|
|     50|    11|  0.3561796|
|     86|    11| 0.31207186|
|     45|    11| 0.27932703|
|     10|    11|-0.28617597|
|     77|    11|  -2.912345|
+-------+------+-----------+

